In [ ]:
import pandas as pd
from Utils import preprocessing

In [2]:
folder = "./Data/"
# ________________________________________load DEV data ___________________________________________________

#dev_df = pd.read_json(folder + "HS-Brexit_dataset/HS-Brexit_dev.json", orient='index')
#dev_df = pd.read_json(folder + "ConvAbuse_dataset/ConvAbuse_dev.json", orient='index')
#dev_df = pd.read_json(folder + "MD-Agreement_dataset/MD-Agreement_dev.json", orient='index')
dev_df = pd.read_json(folder + "ArMIS_dataset/ArMIS_dev.json", orient='index')

dev_df = preprocessing.get_dataset_labels(dev_df)

In [3]:
dev_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement
1,الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,0,0.67,0.33,0
2,النسويات الفسويات,1,0.00,1.00,1
3,سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,1,0.33,0.67,0
4,وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,1,0.33,0.67,0
5,انت قلتي قلتي النسويه امرأة,0,1.00,0.00,1
...,...,...,...,...,...
137,الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,0,1.00,0.00,1
138,احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,1,0.33,0.67,0
139,اول شوف اخواتك وبعدين تكلم مكان صايعات ... عند...,1,0.33,0.67,0
140,ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,0,1.00,0.00,1


In [4]:
dev_df['sentences'] = dev_df['original_text'].apply(lambda x : preprocessing.adjust_split(preprocessing.split_sentence(x)))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))

dev_df['tokens_lists'] = dev_df['sentences'].apply(lambda x: preprocessing.clear_tokens(preprocessing.sentences_tokenizer(x)))
dev_df['tokens_list']= dev_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [4]:
## ARMIS
dev_df['sentences'] = dev_df['original_text'].apply(lambda x : preprocessing.adjust_split(preprocessing.split_sentence(x)))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))

dev_df['tokens_lists'] = dev_df['sentences'].apply(lambda x: preprocessing.clear_tokens(preprocessing.sentences_tokenizer(x, 'ar')))
dev_df['tokens_list']= dev_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [5]:
from transformers import AutoTokenizer, BertModel
model = BertModel.from_pretrained('bert-base-multilingual-cased', output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [6]:

from collections import OrderedDict
from tqdm import tqdm
from Utils import embeddings

In [7]:

dev_df['lemmi_text']= dev_df['tokens_list'].apply(lambda x: ' '.join(x))

dev_df['tokens'] = ''

for index, row  in tqdm(dev_df.iterrows()):
  tokenized_text = embeddings.text_to_token(row.lemmi_text, tokenizer)
  #print(tokenized_text)
  dev_df.loc[index,'tokens'] = str(tokenized_text)

# Convert entire column to a list (saved as str)
from ast import literal_eval
dev_df['tokens'] = dev_df['tokens'].apply(lambda x: literal_eval(x))

141it [00:00, 2162.55it/s]


In [8]:
dev_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list,lemmi_text,tokens
1,الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,0,0.67,0.33,0,[الي يقرأ اسئلة المعلمات يستغرب انهم يعملون مع...,"[[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يع...","[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يعم...",الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,"[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يعم..."
2,النسويات الفسويات,1,0.00,1.00,1,[النسويات الفسويات],"[[النسويات, الفسويات]]","[النسويات, الفسويات]",النسويات الفسويات,"[النسويات, الفسويات]"
3,سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,1,0.33,0.67,0,[سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحا...,"[[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دور...","[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دورا...",سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,"[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دورا..."
4,وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,1,0.33,0.67,0,[وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ...,"[[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين,...","[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين, ...",وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,"[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين, ..."
5,انت قلتي قلتي النسويه امرأة,0,1.00,0.00,1,[انت قلتي قلتي النسويه امرأة],"[[انت, قلتي, قلتي, النسويه, امرأة]]","[انت, قلتي, قلتي, النسويه, امرأة]",انت قلتي قلتي النسويه امرأة,"[انت, قلتي, قلتي, النسويه, امرأة]"
...,...,...,...,...,...,...,...,...,...,...
137,الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,0,1.00,0.00,1,[الرياضات القتالية حيلة نساء لمواجهة العنف الم...,"[[الرياضات, القتالية, حيلة, نساء, لمواجهة, الع...","[الرياضات, القتالية, حيلة, نساء, لمواجهة, العن...",الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,"[الرياضات, القتالية, حيلة, نساء, لمواجهة, العن..."
138,احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,1,0.33,0.67,0,[احب النوع الفسويات لانها تجيب لحالها المسبه ت...,"[[احب, النوع, الفسويات, لانها, تجيب, لحالها, ا...","[احب, النوع, الفسويات, لانها, تجيب, لحالها, ال...",احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,"[احب, النوع, الفسويات, لانها, تجيب, لحالها, ال..."
139,اول شوف اخواتك وبعدين تكلم مكان صايعات ... عند...,1,0.33,0.67,0,"[اول شوف اخواتك وبعدين تكلم مكان صايعات, عندكم...","[[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات...","[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات,...",اول شوف اخواتك وبعدين تكلم مكان صايعات عندكم ز...,"[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات,..."
140,ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,0,1.00,0.00,1,[ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب ق...,"[[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقو...","[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقوا...",ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,"[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقوا..."


### Identity terms

In [25]:
# BREXIT
identity_terms = ['paki', 'deport', '#trump2016', 'islam', 'obama', 'muslim', 'world', 'illegal', 'invasion', 'white', 'blame', 'cause', 'terrorism', 'might', 'another', 'attack', 'watch', '#remain', 'bad', '#brexitvote']

In [42]:
# ConvABUSE
identity_terms = ['asshole', 'pussy', 'fuck', 'stupid', 'sex', 'idiot', 'kill', 'hate', 'bitch', 'slut','mention', 'dead', 'english', 'nope', 'hungry', 'ever', 'old', 'first', 'meet', 'keep']

In [9]:
# MD-Agreement
identity_terms = ['shitty', 'cunt', 'fuck', 'stfu', 'hitler', 'bullshit', 'fuckin', 'niggas', 'asshole', 'pedophile', 'insurance', 'ballot', 'anger', 'difficult', 'wall', 'cancel', 'across', 'replace', 'company', '#anonymous']

In [10]:
# ArMIS
identity_terms = ['الفسويات', 'وقحات', 'قذرات', 'النار', 'متسلطات', 'النسويات', 'ايش', 'ههه', 'رخيصات', 'وهي','نسويه', 'تقول', 'يقول', 'كانوا', 'لأن', 'العنف', 'النسويه', 'علي', 'المراه', 'يأخذ']

In [11]:
def contains_word(tokens, words):
    return any(word in tokens for word in words)

dev_df['term_present']= dev_df['tokens'].apply(lambda x: contains_word(x, identity_terms))
dev_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list,lemmi_text,tokens,term_present
1,الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,0,0.67,0.33,0,[الي يقرأ اسئلة المعلمات يستغرب انهم يعملون مع...,"[[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يع...","[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يعم...",الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,"[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يعم...",True
2,النسويات الفسويات,1,0.00,1.00,1,[النسويات الفسويات],"[[النسويات, الفسويات]]","[النسويات, الفسويات]",النسويات الفسويات,"[النسويات, الفسويات]",True
3,سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,1,0.33,0.67,0,[سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحا...,"[[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دور...","[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دورا...",سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,"[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دورا...",False
4,وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,1,0.33,0.67,0,[وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ...,"[[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين,...","[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين, ...",وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,"[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين, ...",True
5,انت قلتي قلتي النسويه امرأة,0,1.00,0.00,1,[انت قلتي قلتي النسويه امرأة],"[[انت, قلتي, قلتي, النسويه, امرأة]]","[انت, قلتي, قلتي, النسويه, امرأة]",انت قلتي قلتي النسويه امرأة,"[انت, قلتي, قلتي, النسويه, امرأة]",True
...,...,...,...,...,...,...,...,...,...,...,...
137,الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,0,1.00,0.00,1,[الرياضات القتالية حيلة نساء لمواجهة العنف الم...,"[[الرياضات, القتالية, حيلة, نساء, لمواجهة, الع...","[الرياضات, القتالية, حيلة, نساء, لمواجهة, العن...",الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,"[الرياضات, القتالية, حيلة, نساء, لمواجهة, العن...",True
138,احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,1,0.33,0.67,0,[احب النوع الفسويات لانها تجيب لحالها المسبه ت...,"[[احب, النوع, الفسويات, لانها, تجيب, لحالها, ا...","[احب, النوع, الفسويات, لانها, تجيب, لحالها, ال...",احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,"[احب, النوع, الفسويات, لانها, تجيب, لحالها, ال...",True
139,اول شوف اخواتك وبعدين تكلم مكان صايعات ... عند...,1,0.33,0.67,0,"[اول شوف اخواتك وبعدين تكلم مكان صايعات, عندكم...","[[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات...","[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات,...",اول شوف اخواتك وبعدين تكلم مكان صايعات عندكم ز...,"[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات,...",False
140,ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,0,1.00,0.00,1,[ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب ق...,"[[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقو...","[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقوا...",ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,"[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقوا...",True


In [12]:
dev_df['term_present'].value_counts()

term_present
True     71
False    70
Name: count, dtype: int64

In [13]:
dev_df.loc[dev_df['term_present']== True] 

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list,lemmi_text,tokens,term_present
1,الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,0,0.67,0.33,0,[الي يقرأ اسئلة المعلمات يستغرب انهم يعملون مع...,"[[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يع...","[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يعم...",الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,"[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يعم...",True
2,النسويات الفسويات,1,0.00,1.00,1,[النسويات الفسويات],"[[النسويات, الفسويات]]","[النسويات, الفسويات]",النسويات الفسويات,"[النسويات, الفسويات]",True
4,وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,1,0.33,0.67,0,[وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ...,"[[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين,...","[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين, ...",وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,"[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين, ...",True
5,انت قلتي قلتي النسويه امرأة,0,1.00,0.00,1,[انت قلتي قلتي النسويه امرأة],"[[انت, قلتي, قلتي, النسويه, امرأة]]","[انت, قلتي, قلتي, النسويه, امرأة]",انت قلتي قلتي النسويه امرأة,"[انت, قلتي, قلتي, النسويه, امرأة]",True
6,صادقه تبع لجين بينتقمون ويخلون البنات يطالبون ...,1,0.33,0.67,0,[صادقه تبع لجين بينتقمون ويخلون البنات يطالبون...,"[[صادقه, تبع, لجين, بينتقمون, ويخلون, البنات, ...","[صادقه, تبع, لجين, بينتقمون, ويخلون, البنات, ي...",صادقه تبع لجين بينتقمون ويخلون البنات يطالبون ...,"[صادقه, تبع, لجين, بينتقمون, ويخلون, البنات, ي...",True
...,...,...,...,...,...,...,...,...,...,...,...
133,اللي يعتذر بدون غلط يستاهل يجيه اشوف يتشكى ويق...,0,1.00,0.00,1,[اللي يعتذر بدون غلط يستاهل يجيه اشوف يتشكى وي...,"[[اللي, يعتذر, بدون, غلط, يستاهل, يجيه, اشوف, ...","[اللي, يعتذر, بدون, غلط, يستاهل, يجيه, اشوف, ي...",اللي يعتذر بدون غلط يستاهل يجيه اشوف يتشكى ويق...,"[اللي, يعتذر, بدون, غلط, يستاهل, يجيه, اشوف, ي...",True
135,رخيصات يعززون لبعض,1,0.00,1.00,1,[رخيصات يعززون لبعض],"[[رخيصات, يعززون, لبعض]]","[رخيصات, يعززون, لبعض]",رخيصات يعززون لبعض,"[رخيصات, يعززون, لبعض]",True
137,الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,0,1.00,0.00,1,[الرياضات القتالية حيلة نساء لمواجهة العنف الم...,"[[الرياضات, القتالية, حيلة, نساء, لمواجهة, الع...","[الرياضات, القتالية, حيلة, نساء, لمواجهة, العن...",الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,"[الرياضات, القتالية, حيلة, نساء, لمواجهة, العن...",True
138,احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,1,0.33,0.67,0,[احب النوع الفسويات لانها تجيب لحالها المسبه ت...,"[[احب, النوع, الفسويات, لانها, تجيب, لحالها, ا...","[احب, النوع, الفسويات, لانها, تجيب, لحالها, ال...",احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,"[احب, النوع, الفسويات, لانها, تجيب, لحالها, ال...",True


In [14]:
dev_df['term_present'].value_counts()

term_present
True     71
False    70
Name: count, dtype: int64

### Save Datasets

In [29]:
dev_df.loc[dev_df['term_present']== True].to_csv('Syn_df/brexit_syn.csv', sep='\t', index=False)

In [47]:
dev_df.loc[dev_df['term_present']== True].to_csv('Syn_df/convAbuse_syn.csv', sep='\t', index=False)

In [14]:
dev_df.loc[dev_df['term_present']== True].to_csv('Syn_df/MD-Agreement_syn.csv', sep='\t', index=False)

In [15]:
dev_df.loc[dev_df['term_present']== True].to_csv('Syn_df/ArMIS_syn.csv', sep='\t', index=False)

### Datasets Characteristics

In [1]:
import pandas as pd

In [16]:
syn = pd.read_csv('Syn_df/brexit_syn.csv', sep='\t')

In [7]:
syn.shape

(75, 11)

In [9]:
syn.hard_label.value_counts()

hard_label
0    60
1    15
Name: count, dtype: int64

In [10]:
syn.disagreement.value_counts()

disagreement
0    40
1    35
Name: count, dtype: int64

In [12]:
syn = pd.read_csv('Syn_df/MD-Agreement_syn.csv', sep='\t')
print(syn.shape)
print(syn.hard_label.value_counts())
print(syn.disagreement.value_counts())

(102, 11)
hard_label
1    72
0    30
Name: count, dtype: int64
disagreement
0    65
1    37
Name: count, dtype: int64


In [13]:
syn = pd.read_csv('Syn_df/ConvAbuse_syn.csv', sep='\t')
print(syn.shape)
print(syn.hard_label.value_counts())
print(syn.disagreement.value_counts())

(134, 11)
hard_label
1    72
0    62
Name: count, dtype: int64
disagreement
1    100
0     34
Name: count, dtype: int64


In [15]:
syn = pd.read_csv('Syn_df/ArMIS_syn.csv', sep='\t')
print(syn.shape)
print(syn.hard_label.value_counts())
print(syn.disagreement.value_counts())

(71, 11)
hard_label
0    41
1    30
Name: count, dtype: int64
disagreement
1    45
0    26
Name: count, dtype: int64
